In [1]:
import gym
import time
import numpy as np

In [23]:
env = gym.make('CartPole-v1')
env.reset()

for step in range(100):
    env.render()
    action = env.action_space.sample()
#     print(action)
    # sample: (array([ 0.20307615,  0.24289648, -5.11968664, -5.43760231]), 0.0, True, {})
    observation, reward, done, info = env.step(action)
    time.sleep(0.02)
env.close()

In [3]:
env.step(env.action_space.sample())

(array([-0.24667234,  0.65470702, -4.31367009,  5.02866106]), 0.0, True, {})

In [4]:
env.action_space.n

2

In [5]:
env.observation_space

Box(4,)

In [6]:
action_size = env.action_space.n 

In [8]:
def craete_bins(number_of_bins=10):
    bins_cart_position = np.linspace(-4.8, 4.8, number_of_bins)
    bins_cart_velocity = np.linspace(-5, 5, number_of_bins)
    bins_pole_angle = np.linspace(-0.418, 0.418, number_of_bins)
    bins_pole_angular_velocity = np.linspace(-5, 5, number_of_bins)
    
    bins = np.array([bins_cart_position, bins_cart_velocity, bins_pole_angle, bins_pole_angular_velocity])
    
    return bins

In [9]:
# Return the indices of the bins to which each value in input array belongs.
# Example:
bins_smple = [1,2,3,4,5,6]
np.digitize(4.4, bins_smple)

4

In [14]:
num_bins = 10
all_bins = craete_bins(num_bins)
def digitised_observation(observation):
    digitised_bins = list()
    for i, obs in enumerate(observation):
        digitised_bins.append(np.digitize(obs, all_bins[i]))
        
    return digitised_bins

In [15]:
digitised_observation([-0.24667234,  0.65470702, -4.31367009,  5.02866106])

[5, 6, 0, 10]

In [17]:
# Creating Q-Table
q_table = np.zeros([num_bins, num_bins, num_bins, num_bins, env.action_space.n])

In [18]:
q_table.shape

(10, 10, 10, 10, 2)

In [28]:
q_table[0,0,0,0]

array([0., 0.])

In [19]:
# Hyperparameters
# Exploration vs. Exploitation parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.001             # Exponential decay rate for exploration prob

In [20]:
def epsilon_greedy_action_selection(epsilon, q_table, discrete_state):
    '''
    Returns an action for the agent. Note how it uses a random number to decide on
    exploration versus explotation trade-off.
    '''
    random_number = np.random.random()

    # EXPLOITATION, USE BEST Q(s,a) Value
    if random_number > epsilon:
        # Action row for a particular state
        state_row = q_table[discrete_state,:]
        # Index of highest action for state
        # Recall action is mapped to index (e.g. 0=LEFT, 1=DOWN, etc..)
        action = np.argmax(state_row)

    # EXPLORATION, USE A RANDOM ACTION
    else:
        # Return a random 0,1,2,3 action
        action = env.action_space.sample()

    return action

In [21]:
env.reset()

array([ 0.04432948,  0.01639647,  0.04121079, -0.00186996])

In [29]:
env.step(1)

(array([ 0.003329  ,  0.19782752, -0.03872264, -0.26680626]), 1.0, False, {})

In [24]:
digitised_observation(env.reset())

[5, 5, 5, 5]

In [ ]:
# Training Process
epoch = 200
for episdoe in range(epoch):
    state = env.reset()
    done = False
    Total_Reward = 0
    digitised_state = digitised_observation(state)
    while not done:
        action = epsilon_greedy_action_selection(epsilon, q_table, digitised_state)
        new_state, reward, done, info = env.step(action)
        digitised_new_state = digitised_observation(new_state)
        old_q = q_table[digitised_new_state, action]
        
        # Look up current/old qtable value Q(s_t,a_t)
        old_q_value =  q_table[digitised_state,action]  

        # Get the next optimal Q-Value
        next_optimal_q_value = np.max(q_table[digitised_new_state, :])  
        